In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pre_df = pd.read_csv('../data/cleaned/cleaned_vessel_calls.csv')

In [3]:
# preprocess data
# sort by imo number
pre_df.sort_values(by=["IMO", "arrivalDate"], inplace=True)
pre_df['next_unlocode'] = pre_df.groupby('IMO')['place.unlocode'].shift(-1)
pre_df['is_last_imo'] = pre_df['IMO'] != pre_df['IMO'].shift(-1)
pre_df = pre_df[~pre_df["is_last_imo"]]
pre_df = pre_df[pre_df["Event_Type"] != "ARRIVAL"]

In [5]:
pre_df = pre_df.drop(columns=["is_last_imo", "Event_Type", "Timestamp"])
pre_df

,arrivalDate,sailingDate,place.placeName,place.unlocode,place.placeType,place.countryName,IMO,vessel.vesselName,vessel.vesselType,next_unlocode
314761,2023-01-18T14:10:05,2023-01-26T04:05:00,Los Angeles,USLAX,Port,United States of America,1026764,Max 08,Vessel Type (unspecified),CNJGY
314763,2023-02-22T07:17:58,2023-02-23T18:56:16,Jiangyin,CNJGY,Port,People's Republic of China,1026764,Max 08,Vessel Type (unspecified),CNTXG
314765,2023-03-02T16:00:39,2023-03-03T12:31:09,Tianjin Xingang Port,CNTXG,Port,People's Republic of China,1026764,Max 08,Vessel Type (unspecified),CNQAW
314767,2023-03-04T21:02:13,2023-03-05T11:45:29,Qianwan,CNQAW,Sub Port,People's Republic of China,1026764,Max 08,Vessel Type (unspecified),CNMSN
314769,2023-03-06T23:30:18,2023-03-08T00:00:46,Meishan,CNMSN,Sub Port,People's Republic of China,1026764,Max 08,Vessel Type (unspecified),CNNSA
...,...,...,...,...,...,...,...,...,...,...
12569,2023-09-05T05:50:48,2023-09-06T02:46:03,Port Klang (Pelabuhan Klang),MYPKG,Port,Malaysia,9951135,Asterios,fully cellular containership,INMAA
12571,2023-09-10T16:43:35,2023-09-11T20:02:20,Chennai (ex Madras),INMAA,Port,India,9951135,Asterios,fully cellular containership,BDCGP
12573,2023-09-15T05:31:56,2023-09-17T05:28:29,Chittagong (Chattogram),BDCGP,Port,Bangladesh,9951135,Asterios,fully cellular containership,THLCH
12575,2023-09-24T15:15:36,2023-09-25T08:23:38,Laem Chabang,THLCH,Port,Thailand,9951135,Asterios,fully cellular containership,SGSIN
